In [ ]:
import mysql.connector
import pandas as pd
import numpy as np
import rdkit
import re

from rdkit import Chem
from rdkit.Chem.MolStandardize import rdMolStandardize
from rdkit.Chem.rdMolDescriptors import CalcMolFormula
from IPython.display import SVG
from IPython.display import HTML
from rdkit.Chem.Descriptors import MolWt
from rdkit.Chem import Fragments

pd.set_option('display.max_rows', 1000)
pd.set_option('display.width', 1000)
from datetime import datetime
from standardization import standardize_compounds

# Create connection to MySQL server

In [ ]:
cnx = mysql.connector.connect(user='', password='',
                              host='127.0.0.1',
                              database='chembl29')#username, password, database name

# Retrieving assays/targets information related to Nrf2 from ChEMBL

In [ ]:
assay_target_desc_query = ('select * from assays B \
                           join target_dictionary T on B.tid=T.tid \
                           join docs dc on B.doc_id=dc.doc_id \
                           where B.description like "%nrf2%" or \
                           B.description like "%nfe2l2%" or \
                           B.description like "%nuclear%factor%erythroid%2%related%factor%2%" or \
                           B.assay_organism like "%nrf2%" or \
                           B.assay_organism like "%nfe2l2%" or \
                           B.assay_organism like "%nuclear%factor%erythroid%2%related%factor%2%" or \
                           T.organism like "%nrf2%" or \
                           T.organism like "%nfe2l2%" or \
                           T.organism like "%nuclear%factor%erythroid%2%related%factor%2%" or \
                           T.pref_name like "%nrf2%" or \
                           T.pref_name like "%nfe2l2%" or \
                           T.pref_name like "%nuclear%factor%erythroid%2%related%factor%2%";')

In [ ]:
ass_df = pd.read_sql(assay_target_desc_query, cnx)

In [ ]:
ass_df['assay_id'].nunique()

In [ ]:
ass_df.head()

# Retrieving bioactivity/compound information from ChEMBL

In [ ]:
ass_lst = str(ass_df.assay_id.tolist()).replace("[", '')
ass_lst = ass_lst.replace("]", '')

In [ ]:
activity_comp_query = ('select * from assays B \
                           join activities A on B.assay_id=A.assay_id \
                           join compound_structures cs on A.molregno=cs.molregno \
                           where B.assay_id in ' + '(' + ass_lst + ');')

In [ ]:
stract_df = pd.read_sql(activity_comp_query, cnx)

In [ ]:
stract_df.shape

In [ ]:
stract_df = stract_df[~stract_df['canonical_smiles'].isna()]

In [ ]:
df_std = standardize_compounds(list(stract_df['canonical_smiles'].unique()), list(stract_df['molregno'].unique()), "Remove")

In [ ]:
df_std.rename(columns={'cid': 'molregno'}, inplace=True)

In [ ]:
df_std.shape[0] - df_std['cid'].nunique()  # checking whether there were entries, that have been parsed to several seperate lines

In [ ]:
df_std['rdkit_smi'] = df_std['molecule'].apply(lambda x: Chem.MolToSmiles(x))

In [ ]:
stract_df = pd.merge(stract_df, df_std, on='molregno', how='left')
stract_df['std_formula'] = stract_df['molecule'].apply(lambda x: CalcMolFormula(x))

# Filtering non-CHO formulae

In [ ]:
stract_df['CHO'] = stract_df['std_formula'].apply(lambda x: check_cho(x))
stract_df['CHO'] = stract_df['CHO'].fillna(1)

In [ ]:
stract_df['CHO'].value_counts()

In [ ]:
stract_df_cho = stract_df.loc[(stract_df['CHO']==1)]

In [ ]:
stract_df.loc[(stract_df['CHO']==1)&stract_df['description'].str.contains('cytotoxicity')]['description'].unique()

In [ ]:
ass_cytotx_ls = [737190]

In [ ]:
stract_df_cho = stract_df_cho.loc[~stract_df_cho['assay_id'].iloc[:, 0].isin(ass_cytotx_ls)]

In [ ]:
stract_df_cho['activity_comment'].unique()

In [ ]:
stract_df_cho['active'] = np.nan
stract_df_cho = stract_df_cho.loc[~stract_df_cho['activity_comment'].isin(['Non-toxic', 'Non-Toxic', 'Toxic', 'Nd(Toxic)', 'Not Determined'])]
stract_df_cho.loc[stract_df_cho['activity_comment'].isin(['Not Active', 'inactive']), 'active'] = 0
stract_df_cho.loc[stract_df_cho['activity_comment'].isin(['Active', 'Dose-dependent effect', 'tde', 'inconclusive',
                                                 'active', 'Tde']), 'active'] = 1
stract_df_cho.loc[stract_df_cho['standard_type'].isin(['EC50', 'IC50'])&stract_df_cho['standard_relation']=='>', 'active'] = 0
stract_df_cho.loc[stract_df_cho['standard_type'].isin(['Activity'])&(stract_df_cho['standard_units']=='%')&(stract_df_cho['standard_value']<50), 'active'] = 0

In [ ]:
stract_df_cho.loc[stract_df_cho['active'].isna(), 'active'] = 1

In [ ]:
stract_df.loc[stract_df['activity_comment'].isna(), 'activity_comment'] = 'empty'
stract_df['activity_comment'] = stract_df['activity_comment'].astype(str)

In [ ]:
df_fin = stract_df_cho.loc[(stract_df['CHO']==1)].groupby(['molecule_smi'])['active'].mean().reset_index()

In [ ]:
df_fin['active'].value_counts()

In [ ]:
df_fin = df_fin[~(df_fin['active']==0)]

In [ ]:
df_fin.to_csv('./Structures_nrf2_4DL_fine_tuning.csv', index=False)